In [3]:
import torch
from torch import nn

import sys

sys.path.append("..")

from d2lzh_pytorch.myUtils import Timer, Accumulator, try_gpu, try_all_gpus
from d2lzh_pytorch.myPolt import Animator

from d2lzh_pytorch.nlp.model.BERT_model import BERTModel


In [5]:
len_vocab = 20256

net = BERTModel(len_vocab,
                num_hiddens=128,
                norm_shape=[128],
                ffn_num_input=128,
                ffn_num_hiddens=256,
                num_heads=2,
                num_layers=2,
                dropout=0.2,
                key_size=128,
                query_size=128,
                value_size=128,
                hid_in_features=128,
                mlm_in_features=128,
                nsp_in_features=128)

In [8]:
from torchsummary import summary

net = net.to("cuda")
a8 = (3, 255, 255)
summary(net, input_size=a8)


Layer (type:depth-idx)                        Param #
├─BERTEncoder: 1-1                            --
|    └─Embedding: 2-1                         2,592,768
|    └─Embedding: 2-2                         256
|    └─Sequential: 2-3                        --
|    |    └─EncoderBlock: 3-1                 132,480
|    |    └─EncoderBlock: 3-2                 132,480
├─Sequential: 1-2                             --
|    └─Linear: 2-4                            16,512
|    └─Tanh: 2-5                              --
├─MaskLM: 1-3                                 --
|    └─Sequential: 2-6                        --
|    |    └─Linear: 3-3                       16,512
|    |    └─ReLU: 3-4                         --
|    |    └─LayerNorm: 3-5                    256
|    |    └─Linear: 3-6                       2,613,024
├─NextSentencePred: 1-4                       --
|    └─Linear: 2-7                            258
Total params: 5,504,546
Trainable params: 5,504,546
Non-trainable params: 0


Layer (type:depth-idx)                        Param #
├─BERTEncoder: 1-1                            --
|    └─Embedding: 2-1                         2,592,768
|    └─Embedding: 2-2                         256
|    └─Sequential: 2-3                        --
|    |    └─EncoderBlock: 3-1                 132,480
|    |    └─EncoderBlock: 3-2                 132,480
├─Sequential: 1-2                             --
|    └─Linear: 2-4                            16,512
|    └─Tanh: 2-5                              --
├─MaskLM: 1-3                                 --
|    └─Sequential: 2-6                        --
|    |    └─Linear: 3-3                       16,512
|    |    └─ReLU: 3-4                         --
|    |    └─LayerNorm: 3-5                    256
|    |    └─Linear: 3-6                       2,613,024
├─NextSentencePred: 1-4                       --
|    └─Linear: 2-7                            258
Total params: 5,504,546
Trainable params: 5,504,546
Non-trainable params: 0

In [6]:
net

BERTModel(
  (encoder): BERTEncoder(
    (token_embedding): Embedding(20256, 128)
    (segment_embedding): Embedding(2, 128)
    (blks): Sequential(
      (0): EncoderBlock(
        (attention): MultiHeadAttention(
          (attention): DotProductAttention(
            (dropout): Dropout(p=0.2, inplace=False)
          )
          (W_q): Linear(in_features=128, out_features=128, bias=True)
          (W_k): Linear(in_features=128, out_features=128, bias=True)
          (W_v): Linear(in_features=128, out_features=128, bias=True)
          (W_o): Linear(in_features=128, out_features=128, bias=True)
        )
        (addnorm1): AddNorm(
          (dropout): Dropout(p=0.2, inplace=False)
          (ln): LayerNorm((128,), eps=1e-05, elementwise_affine=True)
        )
        (ffn): PositionWiseFFN(
          (dense1): Linear(in_features=128, out_features=256, bias=True)
          (relu): ReLU()
          (dense2): Linear(in_features=256, out_features=128, bias=True)
        )
        (addn